In [13]:
import numpy as np
from ipywidgets import IntSlider, interact, ToggleButtons, HBox, VBox, Output
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, Slider, Label
from bokeh.layouts import widgetbox
from bokeh.models.widgets import RadioButtonGroup, Select
from bokeh.layouts import column, row
from bokeh.models import NumeralTickFormatter, BasicTickFormatter, Range1d, Span, Arrow, OpenHead, NormalHead, VeeHead
from bokeh.io import push_notebook
output_notebook()

p = figure(height=400, width=450, background_fill_color="lightgray")
p.x_range = Range1d(1500,1600)
p.y_range = Range1d(0,1)
p.outline_line_width = 1; p.outline_line_color = "black"; p.min_border_top = 10
p.xaxis.axis_label = "Wavelength (nm)"
p.yaxis.axis_label = "MZI Transmission"
p.xaxis.axis_label_text_font_size = "12pt"
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.axis_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

wl=np.linspace(1400,1700,301)/1000 # wavelength in micron
neff = 2.43
L_up=100 # Length of upper MZM arm in micron
delta_L=25 # Additional length in lower arm
L_low=L_up+delta_L

delta_n=np.linspace(0,6e-4,100)
T=np.absolute(np.exp(1j*2*np.pi*neff*L_up/wl)+np.exp(1j*2*np.pi*neff*L_low/wl))**2/4*0.85
l2=p.line(wl*1000,T,line_color='darkblue')

xl=(1/(1/1.519+1/(4*2.43*25)))*1000
xr=(1/(1/1.519-1/(4*2.43*25)))*1000

vline = Span(location=xl, dimension='height', line_color='red', line_width=1, line_dash='dashed')
vline2 = Span(location=xr, dimension='height', line_color='red', line_width=1, line_dash='dashed')
p.add_layout(vline)
p.add_layout(vline2)

FWHM2 = Arrow(start=VeeHead(size=10), end=VeeHead(size=10), line_color="red", line_dash='dashed',
                   x_start=xl, y_start=0.88, x_end=xr, y_end=0.88)
p.add_layout(FWHM2)
label12=Label(x=1512, y=0.92, text='FWHM', text_color='red', background_fill_color='lightgray')
p.add_layout(label12)

L_up_slider2 =  IntSlider(min=80,max=110,step=5,value=100,description='Upper Arm Length (micron)',
                         style = {'description_width': '200px'}, layout={'width': '450px'})
L_low_slider2 = IntSlider(min=115,max=150,step=5,value=125,description='Lower Arm Length (micron)',
                         style = {'description_width': '200px'}, layout={'width': '450px'})
filter_segments_slider =IntSlider(min=1,max=6, step=1, value=1, description="Number of Filter Segments",
                         style = {'description_width': '200px'}, layout={'width': '450px'})
loss_toggle=ToggleButtons(button_type='success', options=['Low Loss', 'High Loss'], description=' ',
                         tooltips=['Turn Loss On', 'Turn Loss Off'])

def replot2(L_up,L_low,N,loss_select):
#    loss=(1-loss_button_group.active)*0.08
    if loss_select=='On':
        loss=0.08
    else:
        loss=0.0
    l2.data_source.data['y']=np.absolute(np.exp(1j*2*np.pi*neff*L_up/wl)*(1-loss)
                        +np.exp(1j*2*np.pi*neff*L_low/wl)*(1-loss))**(2*N)/4**N
    xl=((1/(1/1.519+1/(4*2.43*(L_low-L_up))))*1000-1519)*(4/np.pi)*np.arccos(0.5**(0.5/N))+1519
    xr=((1/(1/1.519-1/(4*2.43*(L_low-L_up))))*1000-1519)*(4/np.pi)*np.arccos(0.5**(0.5/N))+1519
    FWHM2.x_start=xl
    FWHM2.x_end=xr
    vline.location=xl
    vline2.location=xr
    push_notebook(handle=fig_handle2)

fig_handle2 = show(p,notebook_handle=True)
interact(replot2, L_up=L_up_slider2, L_low=L_low_slider2, N=filter_segments_slider, loss_select=loss_toggle);

Loading BokehJS ...

interactive(children=(IntSlider(value=100, description='Upper Arm Length (micron)', layout=Layout(width='450px…